# <center> <font color=#102C54> <i> Transformers - Trading </font> </center> </i>

<img style="float: left;;" src='https://upload.wikimedia.org/wikipedia/commons/d/db/Logo_ITESO_normal.jpg' width="80" height="160"/></a>
    
<i><center> **Trading Microstructure**  
    <center> Claudia Valeria Chimal Parra 
    <center> Paulo Cesar Ayala Gutiérrez
    <center> Juan Carlos Gutiérrez Valdivia
    <center> Oscar Leonardo Vaca González
    <center> Arturo Espinosa Carabez
        
   <center> May 5th, 2024 

# Classification With Transformers

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ta

In [2]:
data_train = pd.read_csv("./data/aapl_5m_train.csv").dropna()
data_test = pd.read_csv("./data/aapl_5m_test.csv").dropna()

In [3]:
data_train.head()

,Timestamp,Gmtoffset,Datetime,Open,High,Low,Close,Volume
0,1609770600,0,04/01/2021 14:30,133.570007,133.611602,132.389999,132.809997,6624663.0
1,1609770900,0,04/01/2021 14:35,132.750000,132.750000,131.809997,131.889999,2541553.0
2,1609771200,0,04/01/2021 14:40,131.500000,132.339996,131.500000,132.059997,2492415.0
3,1609771500,0,04/01/2021 14:45,132.000000,132.250000,131.899993,132.250000,1859131.0
4,1609771800,0,04/01/2021 14:50,132.000000,132.018096,131.520004,131.589996,1780105.0


## Normalizing Data

In [4]:
train_mean = data_train.loc[:, ["Open", "High", "Low", "Close"]].mean()
train_std = data_train.loc[:, ["Open", "High", "Low", "Close"]].std()

norm_data_train = (data_train.loc[:, ["Open", "High", "Low", "Close"]] - train_mean) / train_std
norm_data_test = (data_test.loc[:, ["Open", "High", "Low", "Close"]] - train_mean) / train_std

## Generating our features

In [5]:
# RSI configurations
rsi_params = {
    'window': [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75]
}

# WMA (Weighted Moving Average) configurations
wma_params = {
    'window': [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75]
}

# MACD configurations
macd_params = {
    'fast_period': [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26],
    'slow_period': [26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40],
    'signal_period': [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
}

# Bollinger Bands configurations
boll_params = {
    'window': [20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34],
    'window_dev': [2, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3, 3.1, 3.2, 3.3, 3.4]
}


In [6]:
# Normalizing
def normalize(data, column_name):
    return (data[column_name] - data[column_name].mean()) / data[column_name].std()

for i in range(15):  # Cambia el rango según el número de configuraciones que desees procesar
    # RSI
    data_train[f'rsi_{i}'] = ta.momentum.RSIIndicator(data_train['Close'], window=rsi_params['window'][i]).rsi()
    data_test[f'rsi_{i}'] = ta.momentum.RSIIndicator(data_test['Close'], window=rsi_params['window'][i]).rsi()

    # WMA
    data_train[f'wma_{i}'] = ta.trend.WMAIndicator(data_train['Close'], window=wma_params['window'][i]).wma()
    data_test[f'wma_{i}'] = ta.trend.WMAIndicator(data_test['Close'], window=wma_params['window'][i]).wma()

    # MACD
    macd = ta.trend.MACD(data_train['Close'], window_fast=macd_params['fast_period'][i], window_slow=macd_params['slow_period'][i], window_sign=macd_params['signal_period'][i])
    data_train[f'macd_{i}'] = macd.macd()
    data_test[f'macd_{i}'] = ta.trend.MACD(data_test['Close'], window_fast=macd_params['fast_period'][i], window_slow=macd_params['slow_period'][i], window_sign=macd_params['signal_period'][i]).macd()

    # Bollinger Bands
    bollinger = ta.volatility.BollingerBands(data_train['Close'], window=boll_params['window'][i], window_dev=boll_params['window_dev'][i])
    data_train[f'bollinger_mavg_{i}'] = bollinger.bollinger_mavg()
    data_train[f'bollinger_hband_{i}'] = bollinger.bollinger_hband()
    data_train[f'bollinger_lband_{i}'] = bollinger.bollinger_lband()

    bollinger_test = ta.volatility.BollingerBands(data_test['Close'], window=boll_params['window'][i], window_dev=boll_params['window_dev'][i])
    data_test[f'bollinger_mavg_{i}'] = bollinger_test.bollinger_mavg()
    data_test[f'bollinger_hband_{i}'] = bollinger_test.bollinger_hband()
    data_test[f'bollinger_lband_{i}'] = bollinger_test.bollinger_lband()

    # Normalize each new indicator and concatenate
    for col in [f'rsi_{i}', f'wma_{i}', f'macd_{i}', f'bollinger_mavg_{i}', f'bollinger_hband_{i}', f'bollinger_lband_{i}']:
        data_train[col] = normalize(data_train, col)
        data_test[col] = normalize(data_test, col)
        norm_data_train = pd.concat([norm_data_train, data_train[col]], axis=1)
        norm_data_test = pd.concat([norm_data_test, data_test[col]], axis=1)


In [7]:
norm_data_train

,Open,High,Low,Close,rsi_0,wma_0,macd_0,bollinger_mavg_0,bollinger_hband_0,bollinger_lband_0,...,macd_13,bollinger_mavg_13,bollinger_hband_13,bollinger_lband_13,rsi_14,wma_14,macd_14,bollinger_mavg_14,bollinger_hband_14,bollinger_lband_14
0,-0.923792,-0.931312,-0.989710,-0.972868,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.976739,-0.986867,-1.027213,-1.032271,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-1.057451,-1.013304,-1.047258,-1.021294,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-1.025166,-1.019107,-1.021394,-1.009026,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-1.025166,-1.034060,-1.045964,-1.051641,-1.601562,-1.026632,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39573,-1.217583,-1.225440,-1.215375,-1.216289,1.556466,-1.220105,0.385375,-1.254636,-1.257028,-1.249399,...,0.040925,-1.253442,-1.290648,-1.203902,0.098348,-1.251961,0.023738,-1.253047,-1.293325,-1.199337
39574,-1.216938,-1.220281,-1.214081,-1.215321,1.595960,-1.218061,0.424076,-1.251647,-1.252429,-1.248044,...,0.081562,-1.253246,-1.289443,-1.204739,0.109625,-1.251084,0.063634,-1.252476,-1.290412,-1.201171
39575,-1.215001,-1.202873,-1.207615,-1.194336,2.075965,-1.209731,0.522100,-1.247511,-1.244231,-1.248023,...,0.148740,-1.252217,-1.282352,-1.209921,0.349348,-1.249640,0.128940,-1.251668,-1.284417,-1.205683
39576,-1.194661,-1.168699,-1.187570,-1.169516,2.262051,-1.195020,0.678735,-1.242295,-1.230995,-1.250923,...,0.244756,-1.250289,-1.268182,-1.220533,0.618022,-1.247560,0.221944,-1.249939,-1.270864,-1.216079


In [8]:
norm_data_train.dropna(inplace=True)
norm_data_test.dropna(inplace=True)

In [9]:
# Export data_train to a CSV file
norm_data_train.to_csv("data_train.csv", index=False)

# Export data_test to a CSV file
norm_data_test.to_csv("data_test.csv", index=False)

In [10]:
norm_data_train = pd.read_csv("data_train.csv")

norm_data_test = pd.read_csv("data_test.csv")

In [11]:
lags = 5

X_train = pd.DataFrame()
X_test = pd.DataFrame()

for lag in range(lags):
    # Add original features with lags
    X_train[f"Close_{lag}"] = norm_data_train.Close.shift(lag)
    
    X_test[f"Close_{lag}"] = norm_data_test.Close.shift(lag)

    # Add indicator features with lags
    for i in range(15): 
        X_train[f'rsi_{i}_{lag}'] = norm_data_train[f'rsi_{i}'].shift(lag)
        X_train[f'macd_{i}_{lag}'] = norm_data_train[f'macd_{i}'].shift(lag)
        X_train[f'bollinger_mavg_{i}_{lag}'] = norm_data_train[f'bollinger_mavg_{i}'].shift(lag)

        X_test[f'rsi_{i}_{lag}'] = norm_data_test[f'rsi_{i}'].shift(lag)
        X_test[f'macd_{i}_{lag}'] = norm_data_test[f'macd_{i}'].shift(lag)
        X_test[f'bollinger_mavg_{i}_{lag}'] = norm_data_test[f'bollinger_mavg_{i}'].shift(lag)

# Train
Y_train = np.where(X_train.Close_0.shift(-5) > X_train.Close_0 * 1.01, 2, 
          np.where(X_train.Close_0.shift(-5) < X_train.Close_0 * 0.99, 1, 0))
Y_train = pd.DataFrame(Y_train)

# Test
Y_test = np.where(X_test.Close_0.shift(-5) > X_test.Close_0 * 1.01, 2, 
         np.where(X_test.Close_0.shift(-5) < X_test.Close_0 * 0.99, 1, 0))
Y_test = pd.DataFrame(Y_test)

# Removing NaNs and the last value due to shifting
#X_train.dropna(inplace=True)
#X_test.dropna(inplace=True)
X_train = X_train.iloc[lags:-1, :].values
X_test = X_test.iloc[lags:-1, :].values

Y_train = Y_train.iloc[lags:-1].values.reshape(-1, 1)
Y_test = Y_test.iloc[lags:-1].values.reshape(-1, 1)

/var/folders/18/2ps3qbw16bg8sch6dyy5c5940000gn/T/ipykernel_57631/1261522803.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train[f'macd_{i}_{lag}'] = norm_data_train[f'macd_{i}'].shift(lag)
/var/folders/18/2ps3qbw16bg8sch6dyy5c5940000gn/T/ipykernel_57631/1261522803.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train[f'bollinger_mavg_{i}_{lag}'] = norm_data_train[f'bollinger_mavg_{i}'].shift(lag)
/var/folders/18/2ps3qbw16bg8sch6dyy5c5940000gn/T/ipykernel_57631/1261522803.py:19: PerformanceWarning: DataFrame is hi

In [12]:
print(X_train.shape)
print(Y_train.shape)

(39080, 230)
(39080, 1)


## Reshaping Tensors

In [13]:
features = X_train.shape[1]

X_train = X_train.reshape(-1, features, 1)
X_test = X_test.reshape(-1, features, 1)

## Classification Model

In [14]:
def create_transformer(inputs, head_size, num_heads, dnn_dim):
    # Stacking layers
    l1 = tf.keras.layers.MultiHeadAttention(key_dim=head_size,
                                            num_heads=num_heads,
                                            dropout=0.2)(inputs, inputs)
    l2 = tf.keras.layers.Dropout(0.2)(l1)
    l3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(l2)
    
    res = l3 + inputs
    
    # Traditional DNN
    l4 = tf.keras.layers.Conv1D(filters=4, kernel_size=1, activation="relu")(res)
    l5 = tf.keras.layers.Dropout(0.2)(l4)
    l6 = tf.keras.layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(l5)
    l7 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(l6)
    return l7 + res

In [15]:
input_shape = X_train.shape[1:]

# Hyperparams
head_size = 256
num_heads = 4
num_transformer_blocks = 4
dnn_dim = 4
units = 128


# Defining input_shape as Input layer
input_layer = tf.keras.layers.Input(input_shape)

# Creating our transformers based on the input layer
transformer_layers = input_layer

for _ in range(num_transformer_blocks):
    # Stacking transformers
    transformer_layers = create_transformer(inputs=transformer_layers,
                                            head_size=head_size,
                                            num_heads=num_heads,
                                            dnn_dim=dnn_dim)

# Adding global pooling
pooling_layer = tf.keras.layers.GlobalAveragePooling1D(data_format="channels_last")\
                                                      (transformer_layers)

# Adding MLP layers
l1 = tf.keras.layers.Dense(units=128, activation="leaky_relu")(pooling_layer)
l2 = tf.keras.layers.Dropout(0.3)(l1)
l3 = tf.keras.layers.Dense(units=128, activation="leaky_relu")(l2)

# Last layer, units = 2 for True and False values
outputs = tf.keras.layers.Dense(units=3, activation="softmax")(l3)

# Model
model = tf.keras.Model(inputs=input_layer,
                       outputs=outputs,
                       name="transformers_classification")

metric = tf.keras.metrics.SparseCategoricalAccuracy()
adam_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
#callbacks = [tf.keras.callbacks.EarlyStopping(monitor="loss",
#                                              patience=10,
#                                              restore_best_weights=True)]

In [16]:
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=adam_optimizer,
    metrics=[metric],
)

In [17]:
model.summary()

Model: "transformers_classification"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 230, 1)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 230, 1)    │      7,169 │ input_layer[0][0… │
│ (MultiHeadAttentio… │                   │            │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 230, 1)    │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 230, 1)    │          2 │ dropout_1[0][0]   │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 230, 1)    │          0 │ layer_normalizat… │
│                     │                   │            │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 230, 4)    │          8 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 230, 4)    │          0 │ conv1d[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 230, 1)    │          5 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 230, 1)    │          2 │ conv1d_1[0][0]    │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 230, 1)    │          0 │ layer_normalizat… │
│                     │                   │            │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 230, 1)    │      7,169 │ add_1[0][0],      │
│ (MultiHeadAttentio… │                   │            │ add_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 230, 1)    │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 230, 1)    │          2 │ dropout_4[0][0]   │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 230, 1)    │          0 │ layer_normalizat… │
│                     │                   │            │ add_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 230, 4)    │          8 │ add_2[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 230, 4)    │          0 │ conv1d_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 230, 1)    │          5 │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 230, 1)    │          2 │ conv1d_3[0][0]    │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 230, 1)    │          0 │ layer_normalizat… │
│                     │                   │            │ add_2[0][0]       │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 45,899 (179.29 KB)

 Trainable params: 45,899 (179.29 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
model.fit(
    X_train,
    Y_train,
    epochs=1,
    batch_size=64,
    #callbacks=callbacks,
)

611/611 ━━━━━━━━━━━━━━━━━━━━ 669s 1s/step - loss: 0.9852 - sparse_categorical_accuracy: 0.5108


In [19]:
model.save("transformer_classifier.keras")

In [20]:
y_hat_train = model.predict(X_train)

1222/1222 ━━━━━━━━━━━━━━━━━━━━ 270s 221ms/step


In [21]:
sum(y_hat_train.argmax(axis=1) == 0)

26

In [22]:
y_hat_test = model.predict(X_test)

605/605 ━━━━━━━━━━━━━━━━━━━━ 134s 221ms/step


In [23]:
sum(y_hat_train.argmax(axis=1) == 1)

0

In [24]:
sum(y_hat_train.argmax(axis=1) == 2)

39054